In [2]:
from deepface import DeepFace
import cv2

def grab_facial_areas(img, detector_backend="opencv", threshold=130, anti_spoofing=True, spoof_threshold=0.8):
    try:
        face_objs = DeepFace.extract_faces(
            img_path=img,
            detector_backend=detector_backend,
            anti_spoofing=anti_spoofing
        )
        faces = []
        for face_obj in face_objs:
            if face_obj["facial_area"]["w"] > threshold:
                is_real = face_obj["is_real"] and (face_obj["antispoof_score"] >= spoof_threshold)
                faces.append((
                    face_obj["facial_area"]["x"],
                    face_obj["facial_area"]["y"],
                    face_obj["facial_area"]["w"],
                    face_obj["facial_area"]["h"],
                    is_real,
                    face_obj["antispoof_score"]
                ))
        return faces
    except Exception as e:
        print(f"Error: {e}")
        return []

def main():
    cap = cv2.VideoCapture(0)
    while True:
        _, img = cap.read()
        faces = grab_facial_areas(img, anti_spoofing=True, spoof_threshold=0.8)  # Adjust threshold here (0.8 = 80%)
        if faces:
            for x, y, w, h, is_real, score in faces:
                color = (0, 255, 0) if is_real else (0, 0, 255)
                label = "Real" if is_real else "Spoof"
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.putText(img, f"{label} - {score:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        cv2.imshow("Anti-Spoofing Demo", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

25-03-30 12:06:46 - Downloading MiniFASNetV2 weights to C:\Users\suryansh/.deepface/weights/2.7_80x80_MiniFASNetV2.pth


Downloading...
From: https://github.com/minivision-ai/Silent-Face-Anti-Spoofing/raw/master/resources/anti_spoof_models/2.7_80x80_MiniFASNetV2.pth
To: C:\Users\suryansh\.deepface\weights\2.7_80x80_MiniFASNetV2.pth
100%|██████████| 1.85M/1.85M [00:00<00:00, 1.90MB/s]


25-03-30 12:06:48 - Downloading MiniFASNetV2 weights to C:\Users\suryansh/.deepface/weights/4_0_0_80x80_MiniFASNetV1SE.pth


Downloading...
From: https://github.com/minivision-ai/Silent-Face-Anti-Spoofing/raw/master/resources/anti_spoof_models/4_0_0_80x80_MiniFASNetV1SE.pth
To: C:\Users\suryansh\.deepface\weights\4_0_0_80x80_MiniFASNetV1SE.pth
100%|██████████| 1.86M/1.86M [00:00<00:00, 2.17MB/s]


Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error: Face could not be detected in numpy array.Please confirm that the picture is a face photo or 

In [5]:
import cv2
import pickle
import numpy as np
from deepface import DeepFace
from face_recognition import face_encodings, face_distance

# Configuration
ENCODINGS_FILE = 'data/encodings.pkl'
SPOOF_THRESHOLD = 0.7  # Minimum score to consider face real
RECOGNITION_THRESHOLD = 0.5  # Face recognition tolerance (lower is stricter)
DETECTOR_BACKEND = "opencv"  # or "retinaface", "mtcnn", etc.

def load_encodings():
    """Load facial encodings from PKL file"""
    try:
        with open(ENCODINGS_FILE, 'rb') as f:
            data = pickle.load(f)
        print(f"Loaded {len(data['encodings'])} encodings for {len(set(data['names']))} people")
        return data['encodings'], data['names']
    except Exception as e:
        print(f"Error loading encodings: {e}")
        return [], []

def extract_face_encodings(face_img):
    """Extract 128D face encodings"""
    rgb_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    encodings = face_encodings(rgb_face)
    return encodings[0] if encodings else None

def recognize_face(face_encoding, known_encodings, known_names):
    """Compare face with known encodings"""
    distances = face_distance(known_encodings, face_encoding)
    best_match_idx = np.argmin(distances)
    min_distance = distances[best_match_idx]
    
    if min_distance <= RECOGNITION_THRESHOLD:
        return known_names[best_match_idx], 1 - min_distance
    return None, None

def main():
    # Load known faces
    known_encodings, known_names = load_encodings()
    if not known_encodings:
        print("No encodings found! Please create encodings first.")
        return

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open video capture")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Couldn't read frame")
            break

        # Detect faces with anti-spoofing
        try:
            face_objs = DeepFace.extract_faces(
                img_path=frame,
                detector_backend=DETECTOR_BACKEND,
                anti_spoofing=True,
                enforce_detection=False
            )
        except Exception as e:
            print(f"Face detection error: {e}")
            continue

        for face_obj in face_objs:
            x, y, w, h = face_obj["facial_area"]["x"], face_obj["facial_area"]["y"], \
                         face_obj["facial_area"]["w"], face_obj["facial_area"]["h"]
            
            # Check if face is real
            is_real = face_obj["is_real"] and (face_obj["antispoof_score"] >= SPOOF_THRESHOLD)
            
            if is_real:
                # Extract face ROI and get encoding
                face_roi = frame[y:y+h, x:x+w]
                face_encoding = extract_face_encodings(face_roi)
                
                if face_encoding is not None:
                    # Recognize face
                    name, confidence = recognize_face(face_encoding, known_encodings, known_names)
                    label = f"{name} ({confidence:.2f})" if name else "Unknown"
                    color = (0, 255, 0)  # Green for recognized
                else:
                    label = "No encoding"
                    color = (0, 255, 255)  # Yellow for detection issues
            else:
                label = f"Spoof ({face_obj['antispoof_score']:.2f})"
                color = (0, 0, 255)  # Red for spoofed

            # Draw bounding box and label
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        cv2.imshow("Secure Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Loaded 250 encodings for 3 people


In [1]:
from deepface import DeepFace
import cv2

def grab_facial_areas(img, detector_backend="opencv", threshold=130, anti_spoofing=False):
    try:
        face_objs = DeepFace.extract_faces(img_path=img, detector_backend=detector_backend, expand_percentage=0, anti_spoofing=anti_spoofing)
        faces = [
            (
                face_obj["facial_area"]["x"],
                face_obj["facial_area"]["y"],
                face_obj["facial_area"]["w"],
                face_obj["facial_area"]["h"],
                face_obj["is_real"],
                face_obj["antispoof_score"]
            )
            for face_obj in face_objs
            if face_obj["facial_area"]["w"] > threshold
        ]
        return faces
    except Exception as e:
        print(str(e))
        return []

def main():
    cap = cv2.VideoCapture(0)
    while True:
        _, img = cap.read()
        faces = grab_facial_areas(img, anti_spoofing=True)
        if faces:
            print(faces)
            for x, y, w, h, is_real, antispoof_score in faces:
                if is_real:
                    color = (0, 255, 0)
                    real = "Real"
                else:
                    color = (0, 0, 255)
                    real = "Spoof"
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.rectangle(img, (x, y+h), (x+w, y+h+30), color, cv2.FILLED)
                cv2.putText(img, f"{real} - {antispoof_score:.2f}%", (x+5, y+h+20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        cv2.imshow("frame", img)
        key = cv2.waitKey(1)
        if key == ord("q"):
            break

if __name__ == "__main__":
    main()

[(284, 106, 193, 193, True, 0.9682137370109558)]
[(279, 101, 205, 205, True, 0.9157362580299377)]
[(282, 103, 197, 197, True, 0.9741723835468292)]
[(287, 110, 185, 185, True, 0.9596004784107208)]
[(279, 98, 205, 205, True, 0.940469890832901)]
Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
[(261, 92, 226, 226, True, 0.9656999707221985)]
[(252, 87, 235, 235, True, 0.975288987159729)]
[(261, 94, 219, 219, True, 0.9884066879749298)]
[(256, 92, 224, 224, True, 0.9683661758899689)]
[(264, 95, 217, 217, True, 0.9687386155128479)]
[(263, 93, 222, 222, True, 0.9743286073207855)]
[(260, 90, 225, 225, True, 0.9737156629562378)]
[(263, 95, 220, 220, True, 0.9716266691684723)]
[(254, 82, 236, 236, True, 0.9738518595695496)]
[(255, 81, 231, 231, True, 0.9172056019306183)]
[(259, 87, 227, 227, True, 0.9700135588645935)]
[(255, 79, 247, 247, True, 0.9825549125671387)]
[(257, 78, 250, 250, True, 0.9765495955

In [2]:
import cv2
import pickle
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from deepface import DeepFace
from face_recognition import face_encodings, face_distance
import os

# Configuration
ENCODINGS_FILE = 'data/encodings.pkl'
ATTENDANCE_FILE = 'data/attendance.csv'
SPOOF_THRESHOLD = 0.7  # Minimum score to consider face real
RECOGNITION_THRESHOLD = 0.5  # Face recognition tolerance (lower is stricter)
DETECTOR_BACKEND = "opencv"  # or "retinaface", "mtcnn", etc.
ATTENDANCE_INTERVAL = timedelta(hours=1)  # Time between allowed attendance marks

def load_encodings():
    """Load facial encodings from PKL file"""
    try:
        with open(ENCODINGS_FILE, 'rb') as f:
            data = pickle.load(f)
        print(f"Loaded {len(data['encodings'])} encodings for {len(set(data['names']))} people")
        return data['encodings'], data['names']
    except Exception as e:
        print(f"Error loading encodings: {e}")
        return [], []

def load_attendance():
    """Load attendance records from CSV"""
    if os.path.exists(ATTENDANCE_FILE):
        df = pd.read_csv(ATTENDANCE_FILE, parse_dates=['timestamp'])
        return df
    return pd.DataFrame(columns=['name', 'timestamp', 'status'])

def save_attendance(df):
    """Save attendance records to CSV"""
    df.to_csv(ATTENDANCE_FILE, index=False)

def can_mark_attendance(name, attendance_df):
    """Check if attendance can be marked (not marked in last hour)"""
    if name is None:
        return False
    
    now = datetime.now()
    last_attendance = attendance_df[attendance_df['name'] == name]
    
    if not last_attendance.empty:
        last_time = pd.to_datetime(last_attendance['timestamp'].iloc[-1])
        return now - last_time >= ATTENDANCE_INTERVAL
    return True

def mark_attendance(name, status, attendance_df):
    """Mark attendance in the dataframe"""
    if name is None:
        return attendance_df
    
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    new_entry = pd.DataFrame([[name, now, status]], 
                            columns=['name', 'timestamp', 'status'])
    return pd.concat([attendance_df, new_entry], ignore_index=True)

def extract_face_encodings(face_img):
    """Extract 128D face encodings"""
    rgb_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    encodings = face_encodings(rgb_face)
    return encodings[0] if encodings else None

def recognize_face(face_encoding, known_encodings, known_names):
    """Compare face with known encodings"""
    distances = face_distance(known_encodings, face_encoding)
    best_match_idx = np.argmin(distances)
    min_distance = distances[best_match_idx]
    
    if min_distance <= RECOGNITION_THRESHOLD:
        return known_names[best_match_idx], 1 - min_distance
    return None, None

def main():
    # Load known faces
    known_encodings, known_names = load_encodings()
    if not known_encodings:
        print("No encodings found! Please create encodings first.")
        return

    # Load attendance records
    attendance_df = load_attendance()
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open video capture")
        return

    # Track recently recognized faces to avoid spamming attendance
    recently_recognized = {}

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Couldn't read frame")
            break

        # Detect faces with anti-spoofing
        try:
            face_objs = DeepFace.extract_faces(
                img_path=frame,
                detector_backend=DETECTOR_BACKEND,
                anti_spoofing=True,
                enforce_detection=False
            )
        except Exception as e:
            print(f"Face detection error: {e}")
            continue

        for face_obj in face_objs:
            x, y, w, h = face_obj["facial_area"]["x"], face_obj["facial_area"]["y"], \
                         face_obj["facial_area"]["w"], face_obj["facial_area"]["h"]
            
            # Check if face is real
            is_real = face_obj["is_real"] and (face_obj["antispoof_score"] >= SPOOF_THRESHOLD)
            
            if is_real:
                # Extract face ROI and get encoding
                face_roi = frame[y:y+h, x:x+w]
                face_encoding = extract_face_encodings(face_roi)
                
                if face_encoding is not None:
                    # Recognize face
                    name, confidence = recognize_face(face_encoding, known_encodings, known_names)
                    label = f"{name} ({confidence:.2f})" if name else "Unknown"
                    color = (0, 255, 0)  # Green for recognized
                    
                    # Check if we should mark attendance
                    if name and name not in recently_recognized:
                        if can_mark_attendance(name, attendance_df):
                            attendance_df = mark_attendance(name, "Present", attendance_df)
                            save_attendance(attendance_df)
                            print(f"Attendance marked for {name}")
                            recently_recognized[name] = datetime.now()
                else:
                    label = "No encoding"
                    color = (0, 255, 255)  # Yellow for detection issues
            else:
                label = f"Spoof ({face_obj['antispoof_score']:.2f})"
                color = (0, 0, 255)  # Red for spoofed
                
                # Mark spoofing attempt in log
                if "spoof_attempt" not in recently_recognized or \
                   (datetime.now() - recently_recognized.get("spoof_attempt", datetime.min)).seconds > 60:
                    attendance_df = mark_attendance("Spoof Attempt", "Spoofed", attendance_df)
                    save_attendance(attendance_df)
                    recently_recognized["spoof_attempt"] = datetime.now()

            # Draw bounding box and label
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Clean up recently recognized dictionary
        current_time = datetime.now()
        recently_recognized = {k: v for k, v in recently_recognized.items() 
                             if current_time - v < timedelta(minutes=5)}

        cv2.imshow("Secure Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Loaded 289 encodings for 3 people
Attendance marked for suryansh
